In [1]:
import findspark
findspark.init('/home/siddharth/spark-2.4.1-bin-hadoop2.7/')

from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('tree_example').getOrCreate()

In [3]:
data = spark.read.csv('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Tree_Methods/College.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [13]:
from pyspark.ml.feature import VectorAssembler,StringIndexer

In [8]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [10]:
output_data = assembler.transform(data)

In [11]:
final_data = output_data.select('features','Private')

In [14]:
indexer = StringIndexer(inputCol='Private',outputCol='Private_index')

In [15]:
final_data = indexer.fit(final_data).transform(final_data)

In [16]:
final_data.show()

+--------------------+-------+-------------+
|            features|Private|Private_index|
+--------------------+-------+-------------+
|[1660.0,1232.0,72...|    Yes|          0.0|
|[2186.0,1924.0,51...|    Yes|          0.0|
|[1428.0,1097.0,33...|    Yes|          0.0|
|[417.0,349.0,137....|    Yes|          0.0|
|[193.0,146.0,55.0...|    Yes|          0.0|
|[587.0,479.0,158....|    Yes|          0.0|
|[353.0,340.0,103....|    Yes|          0.0|
|[1899.0,1720.0,48...|    Yes|          0.0|
|[1038.0,839.0,227...|    Yes|          0.0|
|[582.0,498.0,172....|    Yes|          0.0|
|[1732.0,1425.0,47...|    Yes|          0.0|
|[2652.0,1900.0,48...|    Yes|          0.0|
|[1179.0,780.0,290...|    Yes|          0.0|
|[1267.0,1080.0,38...|    Yes|          0.0|
|[494.0,313.0,157....|    Yes|          0.0|
|[1420.0,1093.0,22...|    Yes|          0.0|
|[4302.0,992.0,418...|    Yes|          0.0|
|[1216.0,908.0,423...|    Yes|          0.0|
|[1130.0,704.0,322...|    Yes|          0.0|
|[3540.0,2

In [17]:
from pyspark.ml.classification import RandomForestClassifier,GBTClassifier,DecisionTreeClassifier

In [18]:
rf = RandomForestClassifier(labelCol='Private_index')
gb = GBTClassifier(labelCol='Private_index')
dt = DecisionTreeClassifier(labelCol='Private_index')

In [19]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [20]:
rf_fit = rf.fit(train_data)
gb_fit = gb.fit(train_data)
dt_fit = dt.fit(train_data)

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [22]:
rf_pred = rf_fit.transform(test_data)
dt_pred = dt_fit.transform(test_data)
gb_pred = gb_fit.transform(test_data)

In [24]:
dt_pred.columns

['features',
 'Private',
 'Private_index',
 'rawPrediction',
 'probability',
 'prediction']

In [25]:
gb_pred.columns

['features',
 'Private',
 'Private_index',
 'rawPrediction',
 'probability',
 'prediction']

In [23]:
rf_pred.columns

['features',
 'Private',
 'Private_index',
 'rawPrediction',
 'probability',
 'prediction']

In [26]:
eval_auc = BinaryClassificationEvaluator(labelCol='Private_index')

In [27]:
rf_auc = eval_auc.evaluate(rf_pred)
rf_auc

0.9867796610169495

In [28]:
dt_auc = eval_auc.evaluate(dt_pred)
dt_auc

0.9160451977401131

In [29]:
gb_auc = eval_auc.evaluate(gb_pred)
gb_auc

0.9439548022598866